In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

## Tuning the hyper-parameters of your estimator
- Train models with different combinations of hyper parameters
- Use cross validation to evaluate the performance of models
- Choose the best one


In [9]:
# loading training/testing data
all_data = pd.read_csv("preprocessed_data.csv")
X = all_data.drop(["Survived"], axis=1)
y = all_data["Survived"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
print(len(X_train))
print(len(X_test))

712
179


In [11]:
model = RandomForestClassifier(max_depth=100, n_estimators=10)
model.fit(X_train, y_train)
predict_y = model.predict(X_train)
print("training accuracy:", (predict_y == y_train).sum()/ len(predict_y))


predict_y = model.predict(X_test)
print("testing accuracy:", (predict_y == y_test).sum()/ len(predict_y))

training accuracy: 0.9691011235955056
testing accuracy: 0.7541899441340782


In [12]:
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.78212291, 0.80337079, 0.83707865, 0.7752809 , 0.83146067])

You can see more classification metics here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [13]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 100,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [14]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [5,10,30,50,100], 'max_depth': [3,5,10,20]}]


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                   scoring='accuracy')
clf.fit(X_train, y_train)

print("best parameters: ", clf.best_params_)
print("best score:", clf.best_score_ )
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])
print(clf.cv_results_["params"])

best parameters:  {'max_depth': 10, 'n_estimators': 50}
best score: 0.8371515808135527
[0.81325717 0.81467547 0.81754161 0.81327686 0.81467547 0.81463607
 0.83297548 0.82452477 0.83148823 0.83152763 0.81045011 0.82729243
 0.83434453 0.83715158 0.82734167 0.80905151 0.80763321 0.82034867
 0.82451492 0.81328671]
[0.02416039 0.03089037 0.0372761  0.02704686 0.02671994 0.02226176
 0.03506637 0.03053389 0.01808125 0.03432513 0.02026991 0.02208604
 0.02815192 0.02671326 0.03288566 0.02672945 0.02158184 0.04261412
 0.03334605 0.02978482]
[{'max_depth': 3, 'n_estimators': 5}, {'max_depth': 3, 'n_estimators': 10}, {'max_depth': 3, 'n_estimators': 30}, {'max_depth': 3, 'n_estimators': 50}, {'max_depth': 3, 'n_estimators': 100}, {'max_depth': 5, 'n_estimators': 5}, {'max_depth': 5, 'n_estimators': 10}, {'max_depth': 5, 'n_estimators': 30}, {'max_depth': 5, 'n_estimators': 50}, {'max_depth': 5, 'n_estimators': 100}, {'max_depth': 10, 'n_estimators': 5}, {'max_depth': 10, 'n_estimators': 10}, {'max

In [15]:
model = RandomForestClassifier(max_depth=10, n_estimators=50)
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.81005587, 0.80337079, 0.85393258, 0.80337079, 0.85393258])

In [10]:
print("Detailed classification report:") 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Detailed classification report:
              precision    recall  f1-score   support

           0       0.75      0.90      0.82       106
           1       0.79      0.58      0.67        73

    accuracy                           0.77       179
   macro avg       0.77      0.74      0.74       179
weighted avg       0.77      0.77      0.76       179


